## LLM Based Classification

In [13]:
VECTOR_DB_DIR = './vector_db'
NUM_RELEVANT_DOCS = 3
EMBEDDING_MODEL = 'paraphrase-MiniLM-L3-v2'
LLM = 'meta/llama-3.1-8b-instruct'

query = "My account login is not working. I've tried resetting my password twice."

Importing the libraries

In [14]:
import os
import sys
import time
from tqdm import tqdm
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.schema import Document
import json
from dotenv import load_dotenv
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.messages import HumanMessage, SystemMessage

Loading API for NVIDIA AI ENDPOINTS

In [15]:
# Loading API Keys
load_dotenv()

True

Importing Data

In [16]:
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

support_tickets = load_json('data/support_tickets.json')
knowledge_base = load_json('data/knowledge_base.json')

In [17]:
# Creating Document from json files

docs = []

for ticket, entry in zip(support_tickets, knowledge_base):
    ticket_text = ticket['text']
    relevant_knowledge = entry['content']
    
    combined_content = f"Support Ticket: {ticket_text}\nRelevant Knowledge: {relevant_knowledge}"
    
    docs.append(combined_content)

# To print the document
for doc in docs:
    print(doc)

Support Ticket: My account login is not working. I've tried resetting my password twice.
Relevant Knowledge: Category 1 - Login Issues - Login issues often occur due to incorrect passwords or account lockouts.
Support Ticket: The app crashes every time I try to upload a photo.
Relevant Knowledge: Category 2 - App Functionality - App crashes can be caused by outdated software or device incompatibility.
Support Ticket: I was charged twice for my last subscription payment.
Relevant Knowledge: Category 3 - Billing - Billing discrepancies may result from processing errors or duplicate transactions.
Support Ticket: I can't find the option to change my profile picture.
Relevant Knowledge: Category 4 - Account Management - Account management includes tasks such as changing profile information, linking social media accounts, and managing privacy settings.
Support Ticket: The video playback is very laggy on my device.
Relevant Knowledge: Category 5 - Performance Issues - Performance issues can b

In [18]:
docs = [Document(page_content=combined_content) for combined_content in docs]

Embedding Model

In [19]:
# Initializing the Embedding Model

embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

/home/architgpt28/miniconda3/envs/EdLight_RAG/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
# creating the vector database and retriever

os.makedirs(VECTOR_DB_DIR, exist_ok=True)
vector_db_path = os.path.join(VECTOR_DB_DIR, EMBEDDING_MODEL.split('/')[-1])

# To check if the vector database exists

if not os.path.exists(vector_db_path):

    print(f'Creating Vector Database at {vector_db_path}')
    Chroma.from_documents(
        documents=docs, 
        embedding=embedding_model, 
        persist_directory=vector_db_path
    )

db = Chroma(
    persist_directory=vector_db_path,
    embedding_function=embedding_model
)

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k":NUM_RELEVANT_DOCS}
)

In [21]:
# retrieving the relevant docs

retrieved_docs = retriever.get_relevant_documents(query)

# To print the retireved chunks of document
for doc in retrieved_docs:
    print(doc.page_content)

Support Ticket: My account login is not working. I've tried resetting my password twice.
Relevant Knowledge: Category 1 - Login Issues - Login issues often occur due to incorrect passwords or account lockouts.
Support Ticket: I can't find the option to change my profile picture.
Relevant Knowledge: Category 4 - Account Management - Account management includes tasks such as changing profile information, linking social media accounts, and managing privacy settings.
Support Ticket: I was charged twice for my last subscription payment.
Relevant Knowledge: Category 3 - Billing - Billing discrepancies may result from processing errors or duplicate transactions.


LLM

In [22]:
# Initializing the LLM using NVIDIA ENDPOINTS

llm = ChatNVIDIA(model = LLM, temperature=0.1, max_tokens=100)

In [23]:
# Generating the prompt for LLM

classification_system_prompt = """You are an assistant for classifying support tickets. Based on the following retrieved support tickets and their corresponding knowledge base categories, classify the given query into the most similar category. Only return the category name and number, nothing else.

Retrieved tickets and categories:
{retrieved_docs}

Query to classify: {query}

Classify the query and respond with only the category name and number. For example, if the query is similar to a login issue, respond with just 'Category 1 - Login Issues'. Only respond from what is mentioned in relevant knowledge and do not generate new text. Only use direct text from relevant knowledge for output."""

messages = [
    SystemMessage(content=classification_system_prompt),
    HumanMessage(content=query)
]

Result

In [24]:
# Invoking the LLM and timming the result

start = time.time()
classification = llm(messages)
end = time.time()

print(classification)

time = end - start
print(f'\nModel Runtime: {time} Seconds')

content='Category 1 - Login Issues' response_metadata={'role': 'assistant', 'content': 'Category 1 - Login Issues', 'token_usage': {'prompt_tokens': 159, 'total_tokens': 165, 'completion_tokens': 6}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-8b-instruct'} id='run-f0417328-438d-4d3b-b5a5-40805dbe935c-0' role='assistant'

Model Runtime: 0.34106922149658203 Seconds
